#Importando bibliotecas

In [87]:
import pandas as pd
from datetime import datetime

#Importando fontes de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#google_drive = '/content/drive/MyDrive/<ADICIONE AQUI O CAMINHO ONDE ESTÃO OS ARQUIVOS>'


In [95]:
amazon_prime_titles  =  pd.read_csv(google_drive + 'amazon_prime_titles.csv', sep=',')
netflix_titles       =  pd.read_csv(google_drive + '/netflix_titles.csv', sep=',')

##Criando Coluna de "Streaming" para identificar origem

In [96]:
amazon_prime_titles['streaming'] = 'amazon_prime'
netflix_titles['streaming']      = 'netflix'

# Unindo tabelas para gerar um único dataset

In [97]:
dados = pd.concat([amazon_prime_titles, netflix_titles], ignore_index=True)

#Transformando e validando dados 

In [98]:
#Transformando coluna 'date_added' para tipo data
dados['date_added'] = pd.to_datetime(dados['date_added'])

#Adicionando uma coluna 'month_added' que contém o mês correspondente de cada 'date_added'
dados['month_added'] = pd.to_datetime(dados['date_added'], format='%m').dt.strftime('%B')

In [102]:
#Verificando duplicidade nos ids Amazon Prime
duplicados_amazon = dados['show_id'][dados['streaming'] == 'amazon_prime'].duplicated().sum()
duplicados_amazon

0

In [101]:
#Verificando duplicidade nos ids Netflix
duplicados_netflix = dados['show_id'][dados['streaming'] == 'netflix'].duplicated().sum()
duplicados_netflix

0

In [103]:
#Verificando campos nulos nos ids Amazon Prime
nulos_amazon =  dados['show_id'][dados['streaming'] == 'amazon_prime'].isna().sum()
nulos_amazon

0

In [104]:
#Verificando campos nulos nos ids Netflix
nulos_netflix = dados['show_id'][dados['streaming'] == 'netflix'].isna().sum()
nulos_netflix

0

In [ ]:
#Verificando campos vazios nos ids Amazon Prime
vazios_amazon = dados['show_id'].isnull().sum()
vazios_amazon

In [106]:
#Verificando campos vazios nos ids Netflix
vazios_netflix = dados['show_id'].isnull().sum()
vazios_netflix

0

In [ ]:
#Verificando tipos de conteúdos
dados['type'].unique()

# Filtrando os dados

##Top 10 atores/atrizes considerando todos os dados

In [107]:
#Criando dataset com dados da coluna cast, tratando os dados, separando-os por nome dos atores/atrizes envolvidos pela vírgula e criando uma linha para cada.
cast = dados['cast'].str.split(',').explode()
cast = cast.reset_index(name="actor/actresses")

In [108]:
#Limpando espaços
cast['actor/actresses'] = cast['actor/actresses'].str.strip()

In [109]:
#Realizando count por nomes e retorno da série é transformado novamente em dataframe
cast = cast.groupby(['actor/actresses']).size().sort_values(ascending=False).head(10).to_frame()

In [110]:
#Renomeando coluna
cast.rename(columns={ cast.columns[0]: "count" }, inplace = True)

In [111]:
#Resetando o index
cast = cast.reset_index()
cast

,actor/actresses,count
0,Anupam Kher,59
1,Maggie Binkley,56
2,Amitabh Bachchan,47
3,Shah Rukh Khan,46
4,Nassar,43
5,Akshay Kumar,41
6,Danny Trejo,39
7,Naseeruddin Shah,39
8,Paresh Rawal,39
9,1,35


##Top 5 países produtores de conteúdos considerando todos os dados e comparando as duas plataformas

In [112]:
# Criando dataset com base na coluna 'country' de dados, separando os nomes dos países por vírgula e criando uma linha para cada
top_5_countries = dados['country'].str.split(',').explode()

#Transformando em df, recebendo 'country' como nome do index
top_5_countries = top_5_countries.reset_index(name="country")

#Removendo espaços
top_5_countries['country'] = top_5_countries['country'].str.strip()

#Realizando count agrupados por países, ordenando os valores pela soma de ocorrências em ambas as plataformas e exibindo os 5 primeiros países
top_5_countries = top_5_countries.groupby(['country']).size().sort_values(ascending=False).head(5).to_frame()

#Renomeando coluna
top_5_countries.rename(columns={ top_5_countries.columns[0]: "count" }, inplace = True)
top_5_countries

,count
country,
United States,4024
India,1292
United Kingdom,873
Canada,480
France,413


In [113]:
# Criando dataset com base na coluna 'country' de dados e filtrando pela coluna streaming apenas netflix, separando os nomes dos países por vírgula e criando uma linha para cada
top_5_countries_netflix = dados['country'].str.split(',').explode()[dados['streaming'] == 'netflix']

#Transformando em df, recebendo 'country' como nome do index
top_5_countries_netflix = top_5_countries_netflix.reset_index(name="country")

#Removendo espaços
top_5_countries_netflix['country'] = top_5_countries_netflix['country'].str.strip()

#Realizando count agrupados por países, ordenando os valores pela soma de ocorrências em ambas as plataformas e exibindo os 5 primeiros países
top_5_countries_netflix = top_5_countries_netflix.groupby(['country']).size().sort_values(ascending=False).head(5).to_frame()

#Renomeando coluna
top_5_countries_netflix.rename(columns={ top_5_countries_netflix.columns[0]: "count" }, inplace = True)

#Criando coluna com rankig do top 5
top_5_countries_netflix.insert(0, 'ranking', range(1, 1 + len(top_5_countries_netflix)))

#Transformando coluna ranking em index
top_5_countries_netflix = top_5_countries_netflix.reset_index()
top_5_countries_netflix = top_5_countries_netflix.set_index('ranking')

top_5_countries_netflix

,country,count
ranking,,
1,United States,3690
2,India,1046
3,United Kingdom,806
4,Canada,445
5,France,393


In [114]:
#Criando dataset com base na coluna 'country' de dados e filtrando pela coluna streaming apenas amazon_prime, separando os nomes dos países por vírgula e criando uma linha para cada
top_5_countries_amazon = dados['country'].str.split(',').explode()[dados['streaming'] == 'amazon_prime']

#Transformando em df, recebendo 'country' como nome do index
top_5_countries_amazon = top_5_countries_amazon.reset_index()

#Removendo espaços
top_5_countries_amazon['country'] = top_5_countries_amazon['country'].str.strip()

#Realizando count agrupados por países, ordenando os valores pela soma de ocorrências em ambas as plataformas e exibindo os 5 primeiros países
top_5_countries_amazon = top_5_countries_amazon.groupby(['country']).size().sort_values(ascending=False).head(5).to_frame()

#Renomeando coluna
top_5_countries_amazon.rename(columns={ top_5_countries_amazon.columns[0]: "count" }, inplace = True)

#Criando coluna com rankig do top 5
top_5_countries_amazon.insert(0, 'ranking', range(1, 1 + len(top_5_countries_netflix)))

#Transformando coluna ranking em index
top_5_countries_amazon = top_5_countries_amazon.reset_index()
top_5_countries_amazon = top_5_countries_amazon.set_index('ranking')


top_5_countries_amazon

,country,count
ranking,,
1,United States,334
2,India,246
3,United Kingdom,67
4,Canada,35
5,France,20


In [115]:
#Combinando os resultados das duas plataformas em um único dataset
top_5_comparison = pd.merge(top_5_countries_netflix, top_5_countries_amazon, on = 'ranking', suffixes=('_netflix', '_amazon_prime'))

# Preenchendo valores nulos com zero
top_5_comparison = top_5_comparison.fillna(0)

top_5_comparison

,country_netflix,count_netflix,country_amazon_prime,count_amazon_prime
ranking,,,,
1,United States,3690,United States,334
2,India,1046,India,246
3,United Kingdom,806,United Kingdom,67
4,Canada,445,Canada,35
5,France,393,France,20


##Mês no qual há mais adições de filmes na plataforma Netflix

In [116]:
# Filtrando por filmes
movies_netflix = dados[(dados['type'] == 'Movie') & (dados['streaming'] == 'netflix')]

In [117]:
#Contando através dos dados agrupados pela coluna 'month_added' e ordenando os dados em ordem decrescente por mês 
movies_month = movies_netflix.groupby('month_added').size().sort_values(ascending=False).head(1).to_frame()

In [118]:
#Renomeando coluna de quantidade
movies_month.rename(columns={ movies_month.columns[0]: "amount" }, inplace = True)
movies_month

,amount
month_added,
July,565


##Quantidade de filmes listados como comédia

In [119]:
#Criando dataset com dados da coluna 'listed_in', tratando os dados. Separando o nome por vírgula e criando uma linha para cada
dados_comedies = dados['listed_in'].str.split(',').explode()[dados['type'] == 'Movie']

#Removendo espaços
dados_comedies = dados_comedies.str.strip()

#Verificando genêros
dados_comedies.unique()

array(['Comedy', 'Drama', 'International', 'Action', 'Suspense',
       'Documentary', 'Fantasy', 'Kids', 'Special Interest',
       'Science Fiction', 'Adventure', 'Horror', 'Sports', 'Arts',
       'Entertainment', 'and Culture', 'Music Videos and Concerts',
       'Fitness', 'Faith and Spirituality', 'Military and War', 'Western',
       'LGBTQ', 'Animation', 'Young Adult Audience', 'Arthouse', 'Anime',
       'Romance', 'Talk Show and Variety', 'Historical', 'Unscripted',
       'Documentaries', 'Children & Family Movies', 'Dramas',
       'Independent Movies', 'International Movies', 'Comedies',
       'Thrillers', 'Romantic Movies', 'Music & Musicals',
       'Horror Movies', 'Sci-Fi & Fantasy', 'Action & Adventure',
       'Classic Movies', 'Anime Features', 'Sports Movies', 'Cult Movies',
       'Faith & Spirituality', 'LGBTQ Movies', 'Stand-Up Comedy',
       'Movies'], dtype=object)

In [120]:
#Criando dataset verificando strings que contenham 'Comed' 
comedy_genres = dados_comedies[(dados_comedies.str.contains('Comed'))]
comedy_genres

0          Comedy
7          Comedy
11         Comedy
15         Comedy
22         Comedy
           ...   
18459    Comedies
18461    Comedies
18466    Comedies
18472    Comedies
18473    Comedies
Name: listed_in, Length: 3838, dtype: object

In [121]:
#Contando quantidade de filmes listado como comédia
dados_comedies = len(comedy_genres)
dados_comedies

3838

##Lista de todos os gêneros de filmes


In [ ]:
#Criando um dataset com filtro "Movie"
dados_movie = dados[dados['type'] == 'Movie']
dados_movie

In [124]:
# Criando uma lista de todos os gêneros de filmes presentes na coluna 'listed_in'
movie_genres = []
for genres_str in dados_movie['listed_in']:
    genres_list = genres_str.split(', ')
    for genre in genres_list:
        if genre not in movie_genres:
            movie_genres.append(genre)

In [125]:
#Transformando em dataset para exibição
movie_genres = pd.DataFrame(movie_genres, columns=['genre'])
movie_genres

,genre
0,Comedy
1,Drama
2,International
3,Action
4,Suspense
5,Documentary
6,Fantasy
7,Kids
8,Special Interest
9,Science Fiction


##A frequência de "TV Show" de todos os dados e comparativamente em relação as duas plataformas


In [126]:
# Calculando a frequência de 'TV Show' 
tv_shows_freq = dados['type'][dados['type'] == 'TV Show'].count()
tv_shows_freq

4530

In [127]:
# Calculando a frequência de 'TV Show' na netflix
tv_shows_freq_netflix = dados['type'][(dados['type'] == 'TV Show') & (dados['streaming'] == 'netflix')].count()
tv_shows_freq_netflix

2676

In [128]:
# Calculando a frequência de 'TV Show' na amazon_prime
tv_shows_freq_amazon = dados['type'][(dados['type'] == 'TV Show') & (dados['streaming'] == 'amazon_prime')].count()
tv_shows_freq_amazon

1854

In [129]:
#Comparando resultado total e das duas plataformas

print(" Frequência total de TV_Show:", str(tv_shows_freq) , "\n Frequência total de TV_Show na Netflix:",str(tv_shows_freq_netflix), "\n Frequência total de TV_Show na Amazon Prime:" ,str(tv_shows_freq_amazon))

 Frequência total de TV_Show: 4530 
 Frequência total de TV_Show na Netflix: 2676 
 Frequência total de TV_Show na Amazon Prime: 1854


##A frequência de "Movies" de todos os dados e comparativamente em relação as duas plataformas

In [130]:
# Calculando a frequência de 'TV Show' 
movie_freq = dados['type'][dados['type'] == 'Movie'].count()

In [131]:
# Calculando a frequência de 'TV Show' na netflix
movie_netflix = dados['type'][(dados['type'] == 'Movie') & (dados['streaming'] == 'netflix')].count()

In [132]:
# Calculando a frequência de 'TV Show' na amazon_prime
movie_freq_amazon = dados['type'][(dados['type'] == 'Movie') & (dados['streaming'] == 'amazon_prime')].count()

In [133]:
#Comparando resultado total e das duas plataformas

print(" Frequência total de Movie:", str(movie_freq) , "\n Frequência total de Movie na Netflix:",str(movie_netflix), "\n Frequência total de Movie na Amazon Prime:" ,str(movie_freq_amazon))

 Frequência total de Movie: 13945 
 Frequência total de Movie na Netflix: 6131 
 Frequência total de Movie na Amazon Prime: 7814
